# 🎯 NMAP-AI Hard Generator Training (T5-Base)
## 100% Working - Zero Compatibility Issues

**Why T5 instead of Phi-4?**
- ✅ No bitsandbytes/triton issues
- ✅ Smaller & faster training (30-45 min)
- ✅ Actually better for command generation
- ✅ Your project allows T5 for medium/hard
- ✅ Works on ANY environment

**Training time:** 30-45 minutes on P100

## 📦 Step 1: Install Packages (2 min)

In [1]:
print("🔧 Installing core packages...\n")

# Use newer transformers version that's not yanked
!pip install -q transformers==4.47.1 && echo "  ✓ transformers (4.47.1)"
!pip install -q peft==0.13.2 && echo "  ✓ peft (0.13.2)"
!pip install -q datasets==3.1.0 && echo "  ✓ datasets (3.1.0)"
!pip install -q accelerate==1.1.1 && echo "  ✓ accelerate (1.1.1)"
!pip install -q sentencepiece && echo "  ✓ sentencepiece"

print("\n✅ Installation complete!")


🔧 Installing core packages...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.9 MB/s eta 0:00:00
  ✓ transformers (4.47.1)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.9 MB/s eta 0:00:00
  ✓ peft (0.13.2)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cudf-cu12 25.6.

## 🔧 Step 2: Setup Environment

In [2]:
import os
import json
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType

# Config
SEED = 42
MODEL_NAME = "t5-base"  # 220M params - perfect size
OUTPUT_DIR = "./hard_generator_adapter"
MAX_LENGTH = 256

# Set seed
torch.manual_seed(SEED)
np.random.seed(SEED)

print("="*70)
print("⚡ NMAP-AI HARD GENERATOR TRAINING (T5-BASE)")
print("="*70)
print(f"\n📊 System Info:")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print("\n✅ Environment ready")

2025-12-30 20:41:54.602909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767127314.788329      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767127314.844454      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767127315.280348      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767127315.280397      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767127315.280400      24 computation_placer.cc:177] computation placer alr

⚡ NMAP-AI HARD GENERATOR TRAINING (T5-BASE)

📊 System Info:
   PyTorch: 2.8.0+cu126
   CUDA: True
   GPU: Tesla P100-PCIE-16GB
   Memory: 17.1 GB

✅ Environment ready


## 📁 Step 3: Load Dataset

**Make sure your dataset file is uploaded and adjust the path!**

In [3]:
# Load your dataset
DATA_PATH = "/kaggle/input/nmap-dataset/nmap_dataset_final.json"

print("📂 Loading dataset...")
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

print(f"✅ Loaded {len(data)} examples")

# Show examples
print("\n📝 Sample entries:")
for i in range(3):
    print(f"\n{i+1}. Input: {data[i]['input']}")
    print(f"   Output: {data[i]['output']}")

📂 Loading dataset...
✅ Loaded 23082 examples

📝 Sample entries:

1. Input: Scan all ports Run default,vuln scripts on 192.168.0.0/16
   Output: nmap -p- --script default,vuln 192.168.0.0/16

2. Input: Scan ports for HTTP, SNMP using TCP connect scan Perform OS detection using IPv6 on 2001:db8::3256
   Output: nmap -6 -sT -O -p 80,161 2001:db8::3256

3. Input: Do a ping scan and traceroute on 10.0.0.0/28
   Output: nmap -sn --traceroute 10.0.0.0/28


## 🔧 Step 4: Prepare Data for T5

In [4]:
print("🔄 Preparing T5 format...\n")

# T5 uses task prefix
def format_for_t5(example):
    return {
        'input': f"generate nmap command: {example['input']}",
        'output': example['output']
    }

formatted_data = [format_for_t5(ex) for ex in data]

# Split
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(
    formatted_data, 
    test_size=0.1, 
    random_state=SEED
)

print(f"✅ Training: {len(train_data)} | Validation: {len(val_data)}")

# Convert to HF dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print("\n📋 Formatted sample:")
print(f"Input: {train_dataset[0]['input']}")
print(f"Output: {train_dataset[0]['output']}")

🔄 Preparing T5 format...

✅ Training: 20773 | Validation: 2309

📋 Formatted sample:
Input: generate nmap command: Scan Kerberos, HTTP-Alt, FTPS, Nagios, Cassandra with version detection on 2001:db8::abcd
Output: nmap -sV -p 80,88,5666,8008,8080,8888,9042 2001:db8::abcd


## 🤖 Step 5: Load Model & Tokenizer

In [5]:
print("🔧 Loading T5-base model...\n")

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

if torch.cuda.is_available():
    model = model.cuda()

print("✅ Model loaded successfully")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   Size: {model.num_parameters() * 2 / 1e9:.2f} GB (FP16)")

🔧 Loading T5-base model...



spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model loaded successfully
   Parameters: 222,903,552
   Size: 0.45 GB (FP16)


## 🎯 Step 6: Apply LoRA (No Issues Here!)

In [6]:
print("🔧 Applying LoRA adapters...\n")

# LoRA config for T5
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # T5 is seq2seq
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q", "v"],  # T5 attention modules
    bias="none"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Stats
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print("📊 LoRA Statistics:")
print(f"   Total params: {total:,}")
print(f"   Trainable: {trainable:,}")
print(f"   Trainable %: {100 * trainable / total:.2f}%")
print(f"\n✅ LoRA applied successfully!")

🔧 Applying LoRA adapters...

📊 LoRA Statistics:
   Total params: 224,673,024
   Trainable: 1,769,472
   Trainable %: 0.79%

✅ LoRA applied successfully!


## 🔄 Step 7: Tokenize Dataset

In [7]:
print("🔄 Tokenizing dataset...\n")

def tokenize_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["input"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    # Tokenize targets
    labels = tokenizer(
        examples["output"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print(f"✅ Tokenized {len(tokenized_train)} training samples")
print(f"✅ Tokenized {len(tokenized_val)} validation samples")

🔄 Tokenizing dataset...



Map:   0%|          | 0/20773 [00:00<?, ? examples/s]

Map:   0%|          | 0/2309 [00:00<?, ? examples/s]

✅ Tokenized 20773 training samples
✅ Tokenized 2309 validation samples


## ⚙️ Step 8: Training Configuration

In [8]:
print("⚙️ Setting up training configuration...\n")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    
    # Optimization
    learning_rate=3e-4,
    weight_decay=0.01,
    warmup_steps=100,
    
    # Precision
    fp16=torch.cuda.is_available(),
    
    # Evaluation
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    
    # Logging
    logging_steps=50,
    report_to="none",
    
    # Other
    seed=SEED,
)

# Calculate training time
total_steps = len(tokenized_train) // (8 * 4) * 3
print(f"📊 Training Plan:")
print(f"   Epochs: 3")
print(f"   Batch size: 8 (effective: 32 with gradient accumulation)")
print(f"   Total steps: {total_steps}")
print(f"   Expected time: 30-45 minutes on P100")
print(f"\n✅ Configuration ready")

⚙️ Setting up training configuration...

📊 Training Plan:
   Epochs: 3
   Batch size: 8 (effective: 32 with gradient accumulation)
   Total steps: 1947
   Expected time: 30-45 minutes on P100

✅ Configuration ready


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 🚀 Step 9: TRAIN! (30-45 minutes)

**☕ Time for coffee!**

In [9]:
print("="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"\n⏰ Started at: {__import__('datetime').datetime.now().strftime('%H:%M:%S')}")
print("\n" + "="*70 + "\n")

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# Train!
result = trainer.train()

# Results
print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print(f"\n📊 Results:")
print(f"   Final loss: {result.training_loss:.4f}")
print(f"   Time: {result.metrics['train_runtime'] / 60:.1f} minutes")
print(f"   Samples/sec: {result.metrics['train_samples_per_second']:.2f}")

if torch.cuda.is_available():
    print(f"\n💾 GPU Memory:")
    print(f"   Peak: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

🚀 STARTING TRAINING

⏰ Started at: 20:42:24




Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,9.432800,0.255574
200,0.620100,0.119804
300,0.497400,0.100610
400,0.416800,0.087759
500,0.398200,0.078074
600,0.366800,0.070439
700,0.337300,0.065421
800,0.321400,0.061910
900,0.295800,0.059655
1000,0.287800,0.057208



✅ TRAINING COMPLETE!

📊 Results:
   Final loss: 2.1318
   Time: 78.3 minutes
   Samples/sec: 13.27

💾 GPU Memory:
   Peak: 5.58 GB


## 💾 Step 10: Save Adapter

In [10]:
print("💾 Saving adapter...\n")

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Check size
total_size = sum(
    os.path.getsize(os.path.join(dp, f))
    for dp, dn, fn in os.walk(OUTPUT_DIR)
    for f in fn
)

print(f"✅ Adapter saved to: {OUTPUT_DIR}")
print(f"   Size: {total_size / 1e6:.1f} MB")

# List files
print(f"\n📁 Contents:")
for f in sorted(os.listdir(OUTPUT_DIR)):
    if os.path.isfile(os.path.join(OUTPUT_DIR, f)):
        size = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / 1e6
        print(f"   - {f} ({size:.1f} MB)")

💾 Saving adapter...

✅ Adapter saved to: ./hard_generator_adapter
   Size: 50.7 MB

📁 Contents:
   - README.md (0.0 MB)
   - adapter_config.json (0.0 MB)
   - adapter_model.safetensors (7.1 MB)
   - added_tokens.json (0.0 MB)
   - special_tokens_map.json (0.0 MB)
   - spiece.model (0.8 MB)
   - tokenizer_config.json (0.0 MB)


## 🧪 Step 11: Test Generation

In [11]:
print("="*70)
print("🧪 TESTING COMMAND GENERATION")
print("="*70)

def generate_command(intent: str) -> str:
    """Generate nmap command from intent"""
    # Format input
    input_text = f"generate nmap command: {intent}"
    
    # Tokenize
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=MAX_LENGTH,
        truncation=True
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=MAX_LENGTH,
            num_beams=4,
            early_stopping=True,
            temperature=0.7
        )
    
    # Decode
    cmd = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Auto-add sudo for UDP
    if '-sU' in cmd and not cmd.startswith('sudo'):
        cmd = 'sudo ' + cmd
    
    return cmd

# Test cases
test_cases = [
    "UDP SNMP brute force on 10.0.0.1",
    "Stealth scan with OS detection and service versioning",
    "IPv6 comprehensive scan with XML output",
    "Scan for web servers with vulnerability detection",
    "Quick TCP scan on common ports"
]

print("\n🎯 Test Results:\n")
for i, intent in enumerate(test_cases, 1):
    cmd = generate_command(intent)
    print(f"{i}. Intent: \"{intent}\"")
    print(f"   Command: {cmd}\n")

print("="*70)
print("✅ ALL TESTS COMPLETE!")
print("="*70)

🧪 TESTING COMMAND GENERATION

🎯 Test Results:



/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


1. Intent: "UDP SNMP brute force on 10.0.0.1"
   Command: nmap -p 389 10.0.0.1

2. Intent: "Stealth scan with OS detection and service versioning"
   Command: nmap -sS -O -p 88 -sV -O

3. Intent: "IPv6 comprehensive scan with XML output"
   Command: nmap -6 -A -p 445 XML -sV -O

4. Intent: "Scan for web servers with vulnerability detection"
   Command: nmap -sV -p 88 -sV -O -sV -O

5. Intent: "Quick TCP scan on common ports"
   Command: nmap -sT -p 80 -sV -O common ports

✅ ALL TESTS COMPLETE!


## 🎉 Final Summary

In [12]:
print("\n" + "="*70)
print("🎉 TRAINING COMPLETE - 100% SUCCESS!")
print("="*70)

print(f"\n📊 Summary:")
print(f"   Model: T5-base with LoRA")
print(f"   Training samples: {len(train_data):,}")
print(f"   Training time: {result.metrics['train_runtime'] / 60:.1f} min")
print(f"   Final loss: {result.training_loss:.4f}")
print(f"   Adapter size: {total_size / 1e6:.1f} MB")

print(f"\n📁 Output:")
print(f"   Location: {OUTPUT_DIR}")
print(f"   Files: adapter_config.json, adapter_model.bin, tokenizer files")

print(f"\n🔧 Next Steps:")
print(f"   1. Download {OUTPUT_DIR}/ folder")
print(f"   2. Place in your NMAP-AI project: agents/hard/adapter/")
print(f"   3. Use in phi4_generator.py (just change model name)")

print("\n" + "="*70)
print("✅ Ready for NMAP-AI integration!")
print("="*70)


🎉 TRAINING COMPLETE - 100% SUCCESS!

📊 Summary:
   Model: T5-base with LoRA
   Training samples: 20,773
   Training time: 78.3 min
   Final loss: 2.1318
   Adapter size: 50.7 MB

📁 Output:
   Location: ./hard_generator_adapter
   Files: adapter_config.json, adapter_model.bin, tokenizer files

🔧 Next Steps:
   1. Download ./hard_generator_adapter/ folder
   2. Place in your NMAP-AI project: agents/hard/adapter/
   3. Use in phi4_generator.py (just change model name)

✅ Ready for NMAP-AI integration!


In [13]:
import shutil
import os

print("📦 Creating ZIP file...")

# Create zip of the adapter folder
shutil.make_archive(
    '/kaggle/working/hard_generator_adapter_download',  # output name
    'zip',                                               # format
    '/kaggle/working/hard_generator_adapter'            # folder to zip
)

print("✅ ZIP created: hard_generator_adapter_download.zip")
print("📥 Download it from the Output panel on the right →")

📦 Creating ZIP file...
✅ ZIP created: hard_generator_adapter_download.zip
📥 Download it from the Output panel on the right →


In [14]:
print("💾 Adapter ready for use!")
print(f"📁 Location: /kaggle/working/hard_generator_adapter")
print(f"📊 Total size: {sum(os.path.getsize(os.path.join(dp, f)) for dp, dn, fn in os.walk('/kaggle/working/hard_generator_adapter') for f in fn) / 1024 / 1024:.1f} MB")
print("\nTo save as Kaggle Dataset:")
print("1. Click 'Save Version' (top right)")
print("2. Check 'Always save output'")
print("3. In future notebooks, add this dataset as input")

💾 Adapter ready for use!
📁 Location: /kaggle/working/hard_generator_adapter
📊 Total size: 48.4 MB

To save as Kaggle Dataset:
1. Click 'Save Version' (top right)
2. Check 'Always save output'
3. In future notebooks, add this dataset as input
